In [1]:
import os
import matplotlib
import matplotlib.colors
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from exp_spec_info import *
from plot_info import *
from select_data import *

In [2]:
# Data pickle path
extended_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\extended_data.pkl"
median_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\median_data.pkl"

# Plot output path
plots_characterization = "C:\\Users\\dosre\\Desktop\\MSCthesis\\thesis\\body_chapters\\chap_6\\images\\characterization"
os.makedirs(plots_characterization, exist_ok=True)

In [3]:
# Load data
extended_data = pd.read_pickle(extended_data_path)
median_data = pd.read_pickle(median_data_path)
conv_traj_data = extended_data[
    (extended_data["solver"] == "FP FP16") |
    (extended_data["solver"] == "FP FP32") |
    (extended_data["solver"] == "FP FP64")
]

In [4]:
relres_diff_data = {
    "FP FP16": [],
    "FP FP32": [],
    "FP FP64": []
}

first_deriv_data = {
    "FP FP16": [],
    "FP FP32": [],
    "FP FP64": []
}

for matrix in df_sel_setup(conv_traj_data, "unprecond")["matrix"].unique():
    for restart_param in df_sel_setup_matrix(conv_traj_data, "unprecond", matrix)["restart_param"].unique():

        sub_data = df_sel_setup_matrix_restart(
            conv_traj_data, "unprecond", matrix, restart_param
        )

        for exp_iter in sub_data["experiment_iter"].unique():

            for solver in ["FP FP16", "FP FP32", "FP FP64"]:

                lowfp_data = sub_data[
                    (sub_data["solver"] == solver) &
                    (sub_data["experiment_iter"] == exp_iter)
                ]

                # Calculate first derivative
                if lowfp_data.size != 0:
                    lowfp_data_row = lowfp_data.iloc[0]["inner_relres"]
                    fm1 = lowfp_data_row[:(lowfp_data_row.size-2)]
                    f1 = lowfp_data_row[2:]
                    deriv_to_add = np.hstack([
                        (lowfp_data_row[1]-lowfp_data_row[0]),
                        0.5*(f1-fm1),
                        (lowfp_data_row[-1]-lowfp_data_row[-2])
                    ])
                    if 15001-deriv_to_add.size > 0:
                        deriv_to_add = np.hstack(
                            (deriv_to_add,
                                np.full(15001-deriv_to_add.size, np.nan))
                        )

                    first_deriv_data[solver].append(deriv_to_add)

                fp64_data = sub_data[
                    (sub_data["solver"] == "FP FP64") &
                    (sub_data["experiment_iter"] == exp_iter)
                ]

                if lowfp_data.size != 0:

                    # Find just comparable data
                    lowfp_data_row = lowfp_data.iloc[0]["inner_relres"]
                    control_data_row = fp64_data.iloc[0]["inner_relres"]
                    if control_data_row.size < lowfp_data_row.size :
                        lowfp_data_row = lowfp_data_row[:control_data_row.size]
                    elif control_data_row.size > lowfp_data_row.size:
                        control_data_row = control_data_row[:lowfp_data_row.size]

                    # Calculate relres diff between lowfp and fp64
                    diff_data_to_add = lowfp_data_row-control_data_row
                    diff_data_to_add /= control_data_row
                    diff_data_to_add[diff_data_to_add < 1e-12] = 1e-12
                    diff_data_to_add = np.append(
                        diff_data_to_add,
                        np.full(15001-diff_data_to_add.size, np.nan)
                    )

                    relres_diff_data[solver].append(diff_data_to_add)


relres_diff_data["FP FP16"] = np.vstack(relres_diff_data["FP FP16"])
relres_diff_data["FP FP32"] = np.vstack(relres_diff_data["FP FP32"])
relres_diff_data["FP FP64"] = np.vstack(relres_diff_data["FP FP64"])

diff_max_val = np.nanmax(np.hstack((relres_diff_data["FP FP16"], relres_diff_data["FP FP32"])))
diff_min_val = np.nanmin(np.hstack((relres_diff_data["FP FP16"], relres_diff_data["FP FP32"])))

first_deriv_data["FP FP16"] = np.vstack(first_deriv_data["FP FP16"])
first_deriv_data["FP FP32"] = np.vstack(first_deriv_data["FP FP32"])
first_deriv_data["FP FP64"] = np.vstack(first_deriv_data["FP FP64"])

deriv_max_val = np.nanmax(np.hstack((first_deriv_data["FP FP16"], first_deriv_data["FP FP32"], first_deriv_data["FP FP64"])))
deriv_min_val = np.nanmin(np.hstack((first_deriv_data["FP FP16"], first_deriv_data["FP FP32"], first_deriv_data["FP FP64"])))

##### 7.1 Wasted Work Characterization Relres Diff Plots

In [8]:
fig, axs = plt.subplots(1, 5, figsize=(8.5, 8.5), width_ratios=[0.245, 0.245, 0.245, 0.245, 0.011])

ax1, ax2, ax3, ax4, ax5 = axs

iter_split = 200
max_iters = max(relres_diff_data["FP FP16"].shape[1], relres_diff_data["FP FP32"].shape[1])
indices_iters = np.arange(0, max_iters, 1)

indices_exp_fp16 = np.arange(0, relres_diff_data["FP FP16"].shape[0], 1)
X_fp16, Y_fp16 = np.meshgrid(indices_iters, indices_exp_fp16)

ax1.pcolormesh(
    X_fp16[:, :iter_split], Y_fp16[:, :iter_split],
    relres_diff_data["FP FP16"][:, :iter_split],
    cmap="RdYlBu_r",
    norm=matplotlib.colors.LogNorm(vmin=diff_min_val, vmax=diff_max_val)
)
ax1.set_xticks(np.arange(0, iter_split+1, 25))
ax1.set_title(f"FP16 (First {iter_split})", fontsize=10)

ax2.pcolormesh(
    X_fp16[:, iter_split:], Y_fp16[:, iter_split:],
    relres_diff_data["FP FP16"][:, iter_split:],
    cmap="RdYlBu_r",
    norm=matplotlib.colors.LogNorm(vmin=diff_min_val, vmax=diff_max_val)
)
ax2.set_xticks([iter_split] + list(np.arange(2500*int(iter_split/2500+1), max_iters+1, 2500)))
ax2.set_title("FP16 (Remainder)", fontsize=10)

indices_exp_fp32 = np.arange(0, relres_diff_data["FP FP32"].shape[0], 1)
X_fp32, Y_fp32 = np.meshgrid(indices_iters, indices_exp_fp32)

ax3.pcolormesh(
    X_fp32[:, :iter_split], Y_fp32[:, :iter_split],
    relres_diff_data["FP FP32"][:, :iter_split],
    cmap="RdYlBu_r",
    norm=matplotlib.colors.LogNorm(vmin=diff_min_val, vmax=diff_max_val)
)
ax3.set_xticks(np.arange(0, iter_split+1, 25))
ax3.set_title(f"FP32 (First {iter_split})", fontsize=10)

ax_im = ax4.pcolormesh(
    X_fp32[:, iter_split:], Y_fp32[:, iter_split:],
    relres_diff_data["FP FP32"][:, iter_split:],
    cmap="RdYlBu_r",
    norm=matplotlib.colors.LogNorm(vmin=diff_min_val, vmax=diff_max_val)
)
ax4.set_xticks([iter_split] + list(np.arange(2500*int(iter_split/2500+1), max_iters+1, 2500)))
ax4.set_title("FP32 (Remainder)", fontsize=10)

for ax in [ax1, ax2, ax3, ax4]:
    ax.set_yticks([])
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.grid()
    ax.set_axisbelow(True)

fig.suptitle("Unpreconditioned Lower Precision FP-GMRES($m$) Relative Residual Fractional Error to FP64")
fig.text(0.5, 0., "Inner Iterations", ha="center")
ax1.set_ylabel("Linear Solve Experiment")
fig.colorbar(ax_im, cax=ax5)

fig.tight_layout(rect=(0., 0.01, 1., 1.))

plt.savefig(
    os.path.join(plots_characterization, f"exp-wasted-work.png"),
    bbox_inches='tight'
)

plt.show()
plt.close()

##### 7.1 Stagnation Characterization

In [6]:
fig, axs = plt.subplots(1, 7, figsize=(9.55, 8.5), width_ratios=[0.11, 0.22, 0.11, 0.22, 0.11, 0.22, 0.01])

ax1, ax2, ax3, ax4, ax5, ax6, ax7 = axs

iter_split = 100
max_iters = max(first_deriv_data["FP FP16"].shape[1], first_deriv_data["FP FP32"].shape[1])
indices_iters = np.arange(0, max_iters, 1)

indices_exp_fp16 = np.arange(0, first_deriv_data["FP FP16"].shape[0], 1)
X_fp16, Y_fp16 = np.meshgrid(indices_iters, indices_exp_fp16)

ax1.pcolormesh(
    X_fp16[:, :iter_split], Y_fp16[:, :iter_split],
    first_deriv_data["FP FP16"][:, :iter_split],
    cmap="RdYlBu_r",
    norm=matplotlib.colors.SymLogNorm(vmin=deriv_min_val, vmax=deriv_max_val, linthresh=1e-12)
)
ax1.set_xticks(np.arange(0, iter_split+1, 25))
ax1.set_title(f"FP16 (First {iter_split})", fontsize=10)
ax1.set_ylabel("Linear Solve Experiment")

ax2.pcolormesh(
    X_fp16[:, iter_split:], Y_fp16[:, iter_split:],
    first_deriv_data["FP FP16"][:, iter_split:],
    cmap="RdYlBu_r",
    norm=matplotlib.colors.SymLogNorm(vmin=deriv_min_val, vmax=deriv_max_val, linthresh=1e-12)
)
ax2.set_xticks([iter_split] + list(np.arange(2500*int(iter_split/2500+1), max_iters+1, 2500)))
ax2.set_title("FP16 (Remainder)", fontsize=10)

indices_exp_fp32 = np.arange(0, first_deriv_data["FP FP32"].shape[0], 1)
X_fp32, Y_fp32 = np.meshgrid(indices_iters, indices_exp_fp32)

ax3.pcolormesh(
    X_fp32[:, :iter_split], Y_fp32[:, :iter_split],
    first_deriv_data["FP FP32"][:, :iter_split],
    cmap="RdYlBu_r",
    norm=matplotlib.colors.SymLogNorm(vmin=deriv_min_val, vmax=deriv_max_val, linthresh=1e-12)
)
ax3.set_xticks(np.arange(0, iter_split+1, 25))
ax3.set_title(f"FP32 (First {iter_split})", fontsize=10)

ax4.pcolormesh(
    X_fp32[:, iter_split:], Y_fp32[:, iter_split:],
    first_deriv_data["FP FP32"][:, iter_split:],
    cmap="RdYlBu_r",
    norm=matplotlib.colors.SymLogNorm(vmin=deriv_min_val, vmax=deriv_max_val, linthresh=1e-12)
)
ax4.set_xticks([iter_split] + list(np.arange(2500*int(iter_split/2500+1), max_iters+1, 2500)))
ax4.set_title("FP32 (Remainder)", fontsize=10)

indices_exp_fp64 = np.arange(0, first_deriv_data["FP FP64"].shape[0], 1)
X_fp64, Y_fp64 = np.meshgrid(indices_iters, indices_exp_fp64)

ax5.pcolormesh(
    X_fp64[:, :iter_split], Y_fp64[:, :iter_split],
    first_deriv_data["FP FP64"][:, :iter_split],
    cmap="RdYlBu_r",
    norm=matplotlib.colors.SymLogNorm(vmin=deriv_min_val, vmax=deriv_max_val, linthresh=1e-12)
)
ax5.set_xticks(np.arange(0, iter_split+1, 25))
ax5.set_title(f"FP64 (First {iter_split})", fontsize=10)

ax_im = ax6.pcolormesh(
    X_fp64[:, iter_split:], Y_fp64[:, iter_split:],
    first_deriv_data["FP FP64"][:, iter_split:],
    cmap="RdYlBu_r",
    norm=matplotlib.colors.SymLogNorm(vmin=deriv_min_val, vmax=deriv_max_val, linthresh=1e-12)
)
ax6.set_xticks([iter_split] + list(np.arange(2500*int(iter_split/2500+1), max_iters+1, 2500)))
ax6.set_title("FP64 (Remainder)", fontsize=10)

for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
    ax.set_yticks([])
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.grid()
    ax.set_axisbelow(True)

fig.suptitle("Unpreconditioned FP-GMRES($m$) Relative Residual First Derivative")
fig.text(0.5, 0., "Inner Iterations", ha="center")
ax1.set_ylabel("Linear Solve Experiment")
fig.colorbar(ax_im, cax=ax7)

fig.tight_layout(rect=(0., 0.01, 1., 1.))

plt.savefig(
    os.path.join(plots_characterization, f"exp-stagnation.png"),
    bbox_inches='tight'
)

plt.show()
plt.close()

##### 7.2 Forgetful Mechanism Characterization

In [24]:
fig, ax = plt.subplots(figsize=(6.4, 4.8), sharex=True, sharey=True)

sub_data = median_data[(median_data["setup"] == "unprecond")]
sub_data = sub_data[["solver", "restart_param", "med_inner_mean_rel_res_frac_err_fp64"]]
sub_data = sub_data[sub_data["solver"].isin(["FP FP16", "FP FP32"])]

mean_rel_res_frac_err_median = sub_data.groupby(["solver", "restart_param"]).apply(
    lambda series: np.median(series)
)
mean_rel_res_frac_err_median.name = "median"
mean_rel_res_frac_err_iqr_25 = sub_data.groupby(["solver", "restart_param"]).apply(
    lambda series: np.percentile(series, 25)
)
mean_rel_res_frac_err_iqr_75 = sub_data.groupby(["solver", "restart_param"]).apply(
    lambda series: np.percentile(series, 75)
)

yerr_low = mean_rel_res_frac_err_median-mean_rel_res_frac_err_iqr_25
yerr_low.name = "yerr_low"

yerr_high = mean_rel_res_frac_err_iqr_75-mean_rel_res_frac_err_median
yerr_high.name = "yerr_high"

plot_data = pd.concat(
    [mean_rel_res_frac_err_median, yerr_low, yerr_high],
    axis=1
)


ax.errorbar(
    plot_data.loc["FP FP16"].index,
    plot_data.loc["FP FP16"]["median"].values,
    yerr=plot_data.loc["FP FP16"][["yerr_low", "yerr_high"]].values.T,
    fmt=".-",
    color=SOLVER_CLR_DICT["FP FP16"],
    capsize=3,
    label="FP FP16"
)

ax.errorbar(
    plot_data.loc["FP FP32"].index,
    plot_data.loc["FP FP32"]["median"].values,
    yerr=plot_data.loc["FP FP32"][["yerr_low", "yerr_high"]].values.T,
    fmt=".-",
    color=SOLVER_CLR_DICT["FP FP32"],
    capsize=3,
    label="FP FP32"
)

ax.grid()
ax.legend()
ax.set_xlabel("Restart Parameter")
ax.set_yscale("symlog", linthresh=plot_data["median"].min())
ax.set_ylabel("Experimental Mean of Rel. Res. Frac. Error")
ax.set_title(f"Lower Precision FP Mean of Rel. Res. Frac. Error from FP64 Across Restart Params")

fig.tight_layout()

plt.savefig(
    os.path.join(plots_characterization, f"exp-forgetful-mechanism.png"),
    bbox_inches='tight'
)

plt.show()
plt.close()

In [17]:
plot_data["median"].min()